# Evaluate a POMDP Policy

In [1]:
rng = MersenneTwister(1);

In [2]:
using AutomotiveDrivingModels
using AutoViz
using GridInterpolations
using POMDPs
using POMDPToolbox
using AutomotivePOMDPs
using QMDP
using SARSOP
using Parameters
using Reel

06-Aug 10:55:41:WARNING:root:replacing docs for 'AutomotivePOMDPs.scene_to_states :: Tuple{Records.Frame{Records.Entity{AutomotiveDrivingModels.VehicleState,AutomotiveDrivingModels.VehicleDef,Int64}},AutomotivePOMDPs.SingleOCPOMDP}' in module 'AutomotivePOMDPs'.
06-Aug 10:55:46:WARNING:root:replacing docs for 'AutomotivePOMDPs.states_to_scene :: Tuple{Dict{Int64,AutomotivePOMDPs.SingleOCState},AutomotivePOMDPs.SingleOCPOMDP}' in module 'AutomotivePOMDPs'.


In [3]:
# include("../src/explicit_pomdps/single_crosswalk/decomposition.jl")

## Define Environment

In [4]:
env = CrosswalkEnv()

AutomotivePOMDPs.CrosswalkEnv(Roadway, AutomotiveDrivingModels.Lane(LaneTag(2, 1), AutomotiveDrivingModels.CurvePt[CurvePt({25.000, -10.000, 1.571}, 0.000, 0.000, NaN), CurvePt({25.000, 10.000, 1.571}, 20.000, 0.000, NaN)], 6.0, AutomotiveDrivingModels.SpeedLimit(-Inf, Inf), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneConnection[], AutomotiveDrivingModels.LaneConnection[]), AutomotiveDrivingModels.ConvexPolygon[ConvexPolygon: len 4 (max 4 pts)
	VecE2(15.000, -1.500)
	VecE2(15.000, -4.500)
	VecE2(21.500, -4.500)
	VecE2(21.500, -1.500)
], AutomotivePOMDPs.CrosswalkParams(2, 50.0, 3.0, 20.0, 6.0, 5.0, 37.0, 8.0, 100, 0.5, 2.0, 10.0))

## Solve Policy in Discrete, Single Agent Environment

In [5]:
dpomdp = SingleOCPOMDP(env=env, ΔT = 0.5, p_birth = 0.3,pos_res = 1.0)
solver = QMDPSolver(max_iterations=100, tolerance=1e-3, verbose=true)
dpolicy = solve(solver, dpomdp);

[Iteration 1   ] residual:          1 | iteration runtime:   2675.730 ms, (      2.68 s total)
[Iteration 2   ] residual:       0.95 | iteration runtime:   2416.366 ms, (      5.09 s total)
[Iteration 3   ] residual:      0.903 | iteration runtime:   2396.772 ms, (      7.49 s total)
[Iteration 4   ] residual:      0.857 | iteration runtime:   2497.444 ms, (      9.99 s total)
[Iteration 5   ] residual:      0.811 | iteration runtime:   2359.723 ms, (      12.3 s total)
[Iteration 6   ] residual:      0.762 | iteration runtime:   2722.061 ms, (      15.1 s total)
[Iteration 7   ] residual:      0.707 | iteration runtime:   3009.113 ms, (      18.1 s total)
[Iteration 8   ] residual:      0.649 | iteration runtime:   2679.910 ms, (      20.8 s total)
[Iteration 9   ] residual:      0.539 | iteration runtime:   2902.553 ms, (      23.7 s total)
[Iteration 10  ] residual:      0.472 | iteration runtime:   2523.947 ms, (      26.2 s total)
[Iteration 11  ] residual:       0.42 | iteration 

## Set up High Fidelity Evaluation Environment

In [20]:
include("../src/eval-crosswalk/config.jl")
include("../src/eval-crosswalk/ego_control.jl")
include("../src/eval-crosswalk/sensor.jl")
include("../src/eval-crosswalk/policy.jl")
include("../src/eval-crosswalk/state_estimation.jl")
include("../src/eval-crosswalk/simulation.jl")
include("../src/eval-crosswalk/baseline_policy.jl")
include("../src/eval-crosswalk/render_helpers.jl")

In [7]:
config = EvalConfig(time_out = 300, n_episodes = 500);

In [8]:
cam = StaticCamera(VecE2(25, 0.), 20.);

In [9]:
sensor = POMDPSensor(pomdp = dpomdp, sensor = SimpleSensor(0.5, 0.5))

POMDPSensor
  pomdp: AutomotivePOMDPs.SingleOCPOMDP
  sensor: SimpleSensor


In [10]:
policy = QMDPEval(env, dpomdp, dpolicy);

In [11]:
up = MixedUpdater(dpomdp, config.sim_dt);

In [12]:
a0 = -4.
ego0 = initial_ego(env, config.rng)
b0 = Dict{Int64, SingleOCDistribution}()
b0[-1] = initial_state_distribution(dpomdp, ego0.state)
o0 = Dict{Int64, SingleOCObs}()

Dict{Int64,AutomotivePOMDPs.SingleOCState} with 0 entries

In [13]:
update_freq = 5
ego_model = CrosswalkDriver(update_freq, 0, env, a0, b0, o0, ego0, sensor, policy, up);

In [14]:
reset_model!(ego_model, ego0)

0.0

In [15]:
overlay = QMDPOverlay(ego_model);

In [16]:
ego0 = initial_ego(env, config.rng)
reset_model!(ego_model, ego0)
models = Dict{Int, DriverModel}()
models[1] = ego_model
scene = initial_scene(models, env, config)
push!(scene, ego0)
nticks = 10
rec = SceneRecord(nticks+1, config.sim_dt)

SceneRecord(nscenes=0)

In [17]:
# execute the simulation
@time simulate!(rec, scene, env, models, nticks, config.rng, config.n_ped, config.callbacks)

  1.726707 seconds (2.83 M allocations: 109.596 MiB, 2.87% gc time)


SceneRecord(nscenes=11)

In [21]:
reset_overlay!(overlay, ego0)
duration, fps, render_rec = animate_record(rec, overlay, 24)
speed_factor = 1
film = roll(render_rec, fps = fps*speed_factor, duration = duration/speed_factor)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmp6qpqoz", 0x000000000000000b, 10.0, nothing)